In [ ]:
import psycopg2
import os
from dotenv import load_dotenv
import pandas as pd
from matplotlib.pyplot import subplots
from bs4 import BeautifulSoup

load_dotenv()

database_password = os.environ.get("DATABASE_PASSWORD")
database_username = os.environ.get("DATABASE_USERNAME")
database_host = os.environ.get("DATABASE_HOST")
database_port = os.environ.get("DATABASE_PORT")
database_name = os.environ.get("DATABASE_NAME")

connection = psycopg2.connect(database=database_name,
                        host=database_host,
                        user=database_username,
                        password=database_password,
                        port=database_port)

In [ ]:
def sql_to_dataframe(conn, query):
    """
    Import data from a PostgreSQL database using a SELECT query
    """
    cursor = conn.cursor()
    try:
        cursor.execute(query)
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error: {error}”")
        cursor.close()
        return 1
    # The execute returns a list of tuples:
    tuples_list = cursor.fetchall()
    colnames = [desc[0] for desc in cursor.description]
    cursor.close()
    # Now we need to transform the list into a pandas DataFrame:
    df = pd.DataFrame(tuples_list, columns=colnames)
    return df

In [ ]:
df = sql_to_dataframe(conn=connection, query="SELECT jcode, COUNT(DISTINCT scrapeid), MIN(checked_time) AS earliest_checked_time, MIN(updated_time) AS earliest_updated_time FROM sitemap_entries GROUP BY jcode")

In [ ]:
df

In [ ]:
df["earliest_updated_time"] = pd.to_datetime(df["earliest_updated_time"], utc=True)

df_counts_by_date = df.groupby(df["earliest_updated_time"].dt.date).size().reset_index(name="count")

In [ ]:
df_counts_by_week = df.groupby(df["earliest_updated_time"].dt.to_period('W')).size().to_timestamp().reset_index(name="count")

In [ ]:
df_counts_by_week

In [ ]:
df_counts_by_week["earliest_updated_time"] = pd.to_datetime(df_counts_by_week["earliest_updated_time"])
df_counts_by_week = df_counts_by_week[df_counts_by_week["earliest_updated_time"] >= pd.to_datetime("2024-01-15")]
df_counts_by_week = df_counts_by_week[df_counts_by_week["earliest_updated_time"] <= pd.to_datetime("2024-03-24")]
df_counts_by_week.set_index("earliest_updated_time", inplace=True)

In [ ]:
df_counts_by_date["earliest_updated_time"] = pd.to_datetime(df_counts_by_date["earliest_updated_time"])
df_counts_by_date = df_counts_by_date[df_counts_by_date["earliest_updated_time"] >= pd.to_datetime("2024-01-11")]
df_counts_by_date.set_index("earliest_updated_time", inplace=True)

In [ ]:
df_counts_by_date = df_counts_by_date.asfreq("D", fill_value=0)
df_counts_by_week = df_counts_by_week.asfreq('W-MON', fill_value=0)

In [ ]:
df_counts_by_week

In [ ]:
fig, ax = subplots(figsize=(10,6))

ax.plot(df_counts_by_date.index, df_counts_by_date["count"])

ax.set_ylim(bottom=0)

ax.set_title("Number of new jobs posted on Civil Service Jobs per day")

ax.set_ylabel("Number of new jobs posted")

fig.subplots_adjust(bottom=0.15)

fig.text(0.5, 0.01, "(Quick, non peer-reviewed visualisation based on unofficial once-daily web scraping. \n Civil Service Jobs data is available under the Open Government Licence v3.0)", ha="center")

In [ ]:
fig, ax = subplots(figsize=(14,6))

ax.plot(df_counts_by_week.index, df_counts_by_week["count"], marker="+", markeredgecolor="red")

ax.set_ylim(bottom=0)

ax.set_title("Number of new jobs posted on Civil Service Jobs per week")

ax.set_ylabel("Number of new jobs posted")

ax.set_xticks(df_counts_by_week.index)

fig.subplots_adjust(bottom=0.15)

fig.text(0.5, 0.01, "(Quick, non peer-reviewed visualisation based on unofficial once-daily web scraping. \n Civil Service Jobs data is available under the Open Government Licence v3.0)", ha="center")